In [1]:
!pip install requests beautifulsoup4 pandas

In [1]:
%cd /content/drive/MyDrive/Data_Mobile

/content/drive/MyDrive/Data_Mobile


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import json
import os
from urllib.parse import urlparse, parse_qs, unquote
import math

# ================= CẤU HÌNH (ĐÃ SỬA) =================
# Sửa thành False để chạy hết tất cả các trang
TEST_MODE = False

# Link này chỉ mang tính tham khảo, logic chính nằm ở API bên dưới
MAIN_URL = "https://didongviet.vn/dien-thoai.html"

DETAIL_SELECTORS = {
    "name": "h1",
    "price_current": ".text-24.font-bold.text-red-500",
    "price_old": "p.line-through",
    "images": "#productDetailImage img",
    "desc": ".box-cmt",
    "variant_item": ".grid.grid-cols-4.gap-2"
}

LIST_SELECTOR = {
    "item_link": "a.item-slider-mobile"
}

# Tạo thư mục lưu ảnh
IMAGE_DIR = "product_images"
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)
# ============================================

def get_soup(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    }
    try:
        response = requests.get(url, headers=headers, timeout=20)
        return BeautifulSoup(response.text, 'html.parser')
    except:
        return None

def parse_specs(soup):
    specs_data = {}
    specs_sections = soup.select(".flex.flex-col.justify-start.items-start.mt-2")

    for section in specs_sections:
        category_tag = section.select_one("p.font-bold")
        if not category_tag:
            continue
        category_name = category_tag.get_text(strip=True)

        specs_data[category_name] = {}
        rows = section.select(".flex.justify-between.items-start")
        for row in rows:
            cols = row.select("div")
            if len(cols) == 2:
                key = cols[0].get_text(strip=True)
                value = cols[1].get_text(strip=True)
                specs_data[category_name][key] = value

    return specs_data

def download_product_images(soup):
    image_files = []
    img_tags = soup.select(DETAIL_SELECTORS["images"])

    for idx, img in enumerate(img_tags):
        src = img.get('src') or img.get('srcset')
        if not src: continue

        try:
            parsed_url = urlparse(src)
            query_params = parse_qs(parsed_url.query)

            if 'url' in query_params:
                real_url = unquote(query_params['url'][0])
            else:
                real_url = src

            filename = os.path.basename(real_url.split('?')[0])
            if not filename or len(filename) < 5:
                filename = f"image_{idx}.jpg"

            img_data = requests.get(real_url).content
            file_path = os.path.join(IMAGE_DIR, filename)

            with open(file_path, 'wb') as handler:
                handler.write(img_data)

            image_files.append(filename)
            # print(f"   📸 Đã tải: {filename}") # Tắt log ảnh cho đỡ rối

        except Exception as e:
            # print(f"   ⚠️ Không thể tải ảnh {idx}: {e}")
            pass

    return " | ".join(image_files)

def scrape_product_detail(url):
    print(f"   ...Đang cào chi tiết: {url}") # Rút gọn log
    soup = get_soup(url)
    if not soup: return None

    try:
        name = soup.select_one(DETAIL_SELECTORS["name"]).get_text(strip=True)

        variants = []
        variant_elements = soup.select(DETAIL_SELECTORS["variant_item"])

        for item in variant_elements:
            v_name = item.get_text(separator=",", strip=True)
            variants.append(v_name)

        variant_str = " | ".join(variants) if variants else "Không có biến thể"

        price_tag = soup.select_one(DETAIL_SELECTORS["price_current"])
        price = price_tag.get_text(strip=True) if price_tag else "Liên hệ"

        price_old_tag = soup.select_one(DETAIL_SELECTORS["price_old"])
        price_old = price_old_tag.get_text(strip=True) if price_old_tag else "Liên hệ"

        specs = parse_specs(soup)
        specs_json = json.dumps(specs, ensure_ascii=False)

        # Lưu ý: Hàm này sẽ tải ảnh về máy, sẽ tốn thời gian
        image_names = download_product_images(soup)

        desc_div = soup.select_one(DETAIL_SELECTORS["desc"])
        description_html = desc_div.decode_contents() if desc_div else ""

        return {
            "name": name,
            "current_price": price,
            "original_price": price_old,
            "variants": variant_str,
            "specs": specs_json,
            "image_names": image_names,
            "description": description_html,
            "url": url,
        }
    except Exception as e:
        print(f"⚠️ Lỗi: {e}")
        return None

def main():
    # Cấu hình API
    limit = 20
    # ID danh mục: 3 là Điện thoại. Nếu muốn cào Tablet/Laptop cần đổi ID hoặc chạy vòng lặp list ID
    category_id = 3
    all_product_links = []

    # 1. Gọi API lần đầu để lấy tổng số
    first_url = f"https://ecomws.didongviet.vn/fe/v1/products?page=1&limit={limit}&category_ids={category_id}"
    try:
        res = requests.get(first_url).json()
        total_products = res['data']['paging']['total']
        total_pages = math.ceil(total_products / limit)
        print(f"📦 Tìm thấy {total_products} sản phẩm. Tổng cộng {total_pages} trang API.")
    except Exception as e:
        print(f"❌ Lỗi API: {e}")
        return

    # 2. Logic chạy vòng lặp
    # Khi TEST_MODE = False, pages_to_run sẽ bằng total_pages (chạy hết)
    pages_to_run = 1 if TEST_MODE else total_pages

    print(f"🔥 Chế độ chạy: {'TEST (1 trang)' if TEST_MODE else 'FULL (Tất cả trang)'}")

    for page in range(1, pages_to_run + 1):
        api_url = f"https://ecomws.didongviet.vn/fe/v1/products?page={page}&limit={limit}&category_ids={category_id}"
        print(f"📡 Đang lấy link trang {page}/{total_pages}...")

        try:
            items = requests.get(api_url).json()['data']['data']
            for item in items:
                # Xử lý trường hợp redirect_url có thể null
                if item.get('redirect_url'):
                    full_link = "https://didongviet.vn/dien-thoai/" + item['redirect_url'] + ".html"
                    all_product_links.append(full_link)
        except Exception as e:
            print(f"Lỗi lấy link trang {page}: {e}")
            continue

    # 3. Cào chi tiết
    results = []
    print(f"🚀 Bắt đầu cào chi tiết {len(all_product_links)} sản phẩm...")
    for idx, link in enumerate(all_product_links):
        print(f"[{idx+1}/{len(all_product_links)}] ", end="") # Hiển thị tiến độ
        data = scrape_product_detail(link)
        if data:
            results.append(data)
        # Giữ thời gian nghỉ để tránh bị chặn IP
        time.sleep(random.uniform(1, 2))

    # 4. Lưu vào CSV
    if results:
        output_file = "didongviet_full_data.csv"
        df = pd.DataFrame(results)
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"✅ Đã lưu thành công {len(results)} dòng vào {output_file}")
    else:
        print("⚠️ Không có dữ liệu để lưu.")

if __name__ == "__main__":
    main()

📦 Tìm thấy 405 sản phẩm. Tổng cộng 21 trang API.
🔥 Chế độ chạy: FULL (Tất cả trang)
📡 Đang lấy link trang 1/21...
📡 Đang lấy link trang 2/21...
📡 Đang lấy link trang 3/21...
📡 Đang lấy link trang 4/21...
📡 Đang lấy link trang 5/21...
📡 Đang lấy link trang 6/21...
📡 Đang lấy link trang 7/21...
📡 Đang lấy link trang 8/21...
📡 Đang lấy link trang 9/21...
📡 Đang lấy link trang 10/21...
📡 Đang lấy link trang 11/21...
📡 Đang lấy link trang 12/21...
📡 Đang lấy link trang 13/21...
📡 Đang lấy link trang 14/21...
📡 Đang lấy link trang 15/21...
📡 Đang lấy link trang 16/21...
📡 Đang lấy link trang 17/21...
📡 Đang lấy link trang 18/21...
📡 Đang lấy link trang 19/21...
📡 Đang lấy link trang 20/21...
📡 Đang lấy link trang 21/21...
🚀 Bắt đầu cào chi tiết 405 sản phẩm...
[1/405]    ...Đang cào chi tiết: https://didongviet.vn/dien-thoai/iphone-12-128gb.html
[2/405]    ...Đang cào chi tiết: https://didongviet.vn/dien-thoai/iphone-11-64gb.html
[3/405]    ...Đang cào chi tiết: https://didongviet.vn/dien-th

In [6]:
import pandas as pd
import random
import re

# 1. Hàm định dạng tiền tệ: 12990990 -> 12.990.990 đ
def format_currency(amount):
    if amount is None or amount == "":
        return ""
    return f"{amount:,}".replace(",", ".") + " đ"

# 2. Hàm làm sạch giá: "20.000.000 đ" -> 20000000
def clean_price(price_str):
    try:
        return int(re.sub(r'[^\d]', '', str(price_str)))
    except:
        return 0

# 3. Hàm tính giá biến thể (+/- 10%, đuôi 990)
def generate_variant_price(base_price):
    delta = base_price * 0.1
    price = random.uniform(base_price - delta, base_price + delta)
    # Chia lấy nguyên cho 1000 để làm tròn hàng nghìn, sau đó cộng 990
    final_price = ((int(price) // 1000) + 990) * 1000
    return final_price

# Đọc dữ liệu
df = pd.read_csv('didongviet_full_data.csv')
final_rows = []

for _, row in df.iterrows():
    base_p_int = clean_price(row['current_price'])

    # --- BƯỚC A: THÊM SẢN PHẨM GỐC (Không có variant price) ---
    original_row = row.to_dict()
    original_row['variant_name'] = 'Sản phẩm gốc'
    original_row['variant_price'] = ""
    final_rows.append(original_row)

    # --- BƯỚC B: TRỘN BIẾN THỂ MÀU SẮC ---
    if pd.notna(row['variants']) and '|' in str(row['variants']):
        colors_part = str(row['variants']).split('|')[1]
        colors = [c.strip() for c in colors_part.split(',')]

        for color in colors:
            v_row = row.to_dict()
            v_row['variant_name'] = f"Màu: {color}"
            v_p_int = generate_variant_price(base_p_int)
            v_row['variant_price'] = format_currency(v_p_int)
            final_rows.append(v_row)

# Tạo DataFrame mới và lưu file
df_final = pd.DataFrame(final_rows)
df_final.to_csv('didongviet_final_processed.csv', index=False, encoding='utf-8-sig')

print("✅ Đã tạo file thành công: didongviet_final_processed.csv")

✅ Đã tạo file thành công: didongviet_final_processed.csv


In [ ]:
import pandas as pd
import random
import re


In [7]:

df = pd.read_csv('didongviet_final_processed.csv')
display(df)

,name,current_price,original_price,variants,specs,image_names,description,url,variant_name,variant_price
0,iPhone 12 128GB Chính hãng (VN/A),12.990.000 đ,Liên hệ,"128GB | Tím,Đen,Xanh dương,Xanh lá cây,Đỏ,Trắng","{""Màn hình"": {""Màn hình rộng"": ""6.1 inches"", ""...",iphone-12-64gb-didongviet.jpeg | iphone-12-den...,"<h2 id=""mua-iphone-12-128gb-chinh-hang-vn-a-2-...",https://didongviet.vn/dien-thoai/iphone-12-128...,Sản phẩm gốc,NaN
1,iPhone 12 128GB Chính hãng (VN/A),12.990.000 đ,Liên hệ,"128GB | Tím,Đen,Xanh dương,Xanh lá cây,Đỏ,Trắng","{""Màn hình"": {""Màn hình rộng"": ""6.1 inches"", ""...",iphone-12-64gb-didongviet.jpeg | iphone-12-den...,"<h2 id=""mua-iphone-12-128gb-chinh-hang-vn-a-2-...",https://didongviet.vn/dien-thoai/iphone-12-128...,Màu: Tím,13.873.000 đ
2,iPhone 12 128GB Chính hãng (VN/A),12.990.000 đ,Liên hệ,"128GB | Tím,Đen,Xanh dương,Xanh lá cây,Đỏ,Trắng","{""Màn hình"": {""Màn hình rộng"": ""6.1 inches"", ""...",iphone-12-64gb-didongviet.jpeg | iphone-12-den...,"<h2 id=""mua-iphone-12-128gb-chinh-hang-vn-a-2-...",https://didongviet.vn/dien-thoai/iphone-12-128...,Màu: Đen,13.664.000 đ
3,iPhone 12 128GB Chính hãng (VN/A),12.990.000 đ,Liên hệ,"128GB | Tím,Đen,Xanh dương,Xanh lá cây,Đỏ,Trắng","{""Màn hình"": {""Màn hình rộng"": ""6.1 inches"", ""...",iphone-12-64gb-didongviet.jpeg | iphone-12-den...,"<h2 id=""mua-iphone-12-128gb-chinh-hang-vn-a-2-...",https://didongviet.vn/dien-thoai/iphone-12-128...,Màu: Xanh dương,14.347.000 đ
4,iPhone 12 128GB Chính hãng (VN/A),12.990.000 đ,Liên hệ,"128GB | Tím,Đen,Xanh dương,Xanh lá cây,Đỏ,Trắng","{""Màn hình"": {""Màn hình rộng"": ""6.1 inches"", ""...",iphone-12-64gb-didongviet.jpeg | iphone-12-den...,"<h2 id=""mua-iphone-12-128gb-chinh-hang-vn-a-2-...",https://didongviet.vn/dien-thoai/iphone-12-128...,Màu: Xanh lá cây,14.578.000 đ
...,...,...,...,...,...,...,...,...,...,...
735,Xiaomi Poco M8 5G Chính Hãng,Liên hệ,Liên hệ,Không có biến thể,{},xiaomi-poco-m8-5g-diddongviet.png | xiaomi-poc...,"<p dir=""ltr""><strong>Xiaomi POCO M8</strong> đ...",https://didongviet.vn/dien-thoai/xiaomi-poco-m...,Sản phẩm gốc,NaN
736,Xiaomi Poco M8 Pro 5G Chính Hãng,Liên hệ,Liên hệ,Không có biến thể,{},xiaomi-poco-m8-pro-5g-didongviet.png | xiaomi-...,"<p style=""text-align: justify;""><strong>POCO M...",https://didongviet.vn/dien-thoai/xiaomi-poco-m...,Sản phẩm gốc,NaN
737,TECNO Spark Go 3 Chính Hãng,Liên hệ,Liên hệ,Không có biến thể,{},tecno-spark-go-3-didongviet.png | tecno-spark-...,"<p dir=""ltr"">TECNO Spark Go 3 là lựa chọn nổi ...",https://didongviet.vn/dien-thoai/tecno-spark-g...,Sản phẩm gốc,NaN
738,OPPO A6t Pro Chính Hãng,Liên hệ,Liên hệ,Không có biến thể,"{""Màn hình"": {""Tần số quét"": ""120Hz"", ""Công ng...",oppo-a6t-pro-didongviet.png | oppo-a6t-pro-did...,"<p dir=""ltr""><a href=""https://didongviet.vn/di...",https://didongviet.vn/dien-thoai/oppo-a6t-pro....,Sản phẩm gốc,NaN


In [5]:
import pandas as pd
import json
import re

# 1. Đọc dữ liệu
file_path = 'didongviet_final_processed.csv'
df = pd.read_csv(file_path)

# --- PHẦN 1: XỬ LÝ GIÁ ---
def clean_price(price_str):
    if pd.isna(price_str) or not isinstance(price_str, str):
        return None
    clean_str = re.sub(r'[^\d]', '', price_str)
    if clean_str:
        return float(clean_str)
    return None

price_cols = ['current_price', 'original_price', 'variant_price']
for col in price_cols:
    df[f'{col}_cleaned'] = df[col].apply(clean_price)

# --- PHẦN 2: TÁCH THÔNG SỐ KỸ THUẬT ---
def extract_specs(specs_str):
    try:
        if pd.isna(specs_str):
            return {}
        spec_dict = json.loads(specs_str)
        extracted = {}
        def get_val(d, *keys):
            for key in keys:
                if isinstance(d, dict) and key in d:
                    d = d[key]
                else:
                    return None
            return d

        extracted['screen_size'] = get_val(spec_dict, 'Màn hình', 'Màn hình rộng')
        extracted['chipset'] = get_val(spec_dict, 'Hệ điều hành & CPU', 'Chip xử lý (CPU)')
        extracted['ram'] = get_val(spec_dict, 'Bộ nhớ & Lưu trữ', 'RAM')
        extracted['storage'] = get_val(spec_dict, 'Bộ nhớ & Lưu trữ', 'Bộ nhớ trong')
        extracted['battery'] = get_val(spec_dict, 'Pin & Sạc', 'Dung lượng pin')
        extracted['os'] = get_val(spec_dict, 'Hệ điều hành & CPU', 'Hệ điều hành')
        return extracted
    except Exception as e:
        return {}

specs_data = df['specs'].apply(extract_specs).apply(pd.Series)
df = pd.concat([df, specs_data], axis=1)

# --- PHẦN 3: LÀM SẠCH MÔ TẢ (DESCRIPTION) ---
def clean_html_regex(html_text):
    if pd.isna(html_text) or not isinstance(html_text, str):
        return ""
    # Xóa các thẻ HTML <...>
    text = re.sub(r'<[^>]+>', ' ', html_text)
    # Xóa khoảng trắng thừa (nhiều dấu cách liền nhau thành 1 dấu cách)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['description_cleaned'] = df['description'].apply(clean_html_regex)

# 4. Chọn các cột cần xuất ra
output_cols = [
    'name',
    'current_price_cleaned', 'original_price_cleaned',
    'variant_name', 'variant_price_cleaned',
    'ram', 'storage', 'chipset', 'screen_size', 'battery', 'os',
    'description_cleaned', # Cột mới đã xử lý
    'url'
]

final_df = df[output_cols]

# 5. Lưu file
final_df.to_csv('dien_thoai_full_info.csv', index=False)

print("Đã xử lý xong! File kết quả là: dien_thoai_full_info.csv")
print(final_df[['name', 'description_cleaned']].head())

Đã xử lý xong! File kết quả là: dien_thoai_full_info.csv
                                name  \
0  iPhone 12 128GB Chính hãng (VN/A)   
1  iPhone 12 128GB Chính hãng (VN/A)   
2  iPhone 12 128GB Chính hãng (VN/A)   
3  iPhone 12 128GB Chính hãng (VN/A)   
4  iPhone 12 128GB Chính hãng (VN/A)   

                                 description_cleaned  
0  Mua iPhone 12 128GB Chính Hãng VN/A - 2 Camera...  
1  Mua iPhone 12 128GB Chính Hãng VN/A - 2 Camera...  
2  Mua iPhone 12 128GB Chính Hãng VN/A - 2 Camera...  
3  Mua iPhone 12 128GB Chính Hãng VN/A - 2 Camera...  
4  Mua iPhone 12 128GB Chính Hãng VN/A - 2 Camera...  
